This notebook will include code to clean, preprocess, and merge the datasets (e.g., handling missing values, merging data based on star/planet identifiers, normalization).

In [1]:
import numpy as np
import pandas as pd

In [15]:
''' Step 1: Load and assess the data '''
# check out any columns that look useless, columns of interest, data type of columns (continous, categorical, timeseries, etc.)

# turning the vot data to csv for GAIA
from astropy.io.votable import parse

votable = parse("../data/gaiaDR3HostStar.vot") # Load the VOTable file

table = votable.get_first_table().to_table() # Convert the first table in the VOTable to an Astropy Table

table.write("../data/gaiaDR3HostStar.csv", format="csv", overwrite=True) # save to csv

# Read the CSV files into a DataFrame
dfGAIA = pd.read_csv('../data/gaiaDR3HostStar.csv')
dfNASAExo = pd.read_csv('../data/NASA_Exoplanet_Archive.csv')
dfDropPHL = pd.read_csv('../data/PHL_Exoplanet_Habitability.csv')

In [16]:
''' Step 2: Clean and combine the data'''
columnsDropGAIA = [] # for Gaia, merge with sourceID 
columnsDropNASAExo = ["koi_disposition", "koi_pdisposition", "koi_score", # not related to habitability
                      "koi_fpflag_nt", "koi_fpflag_ss", "koi_fpflag_co", "koi_fpflag_ec", # not related to habitability
                      "koi_period_err1", "koi_period_err2", 
                      "koi_time0bk_err1", "koi_time0bk_err2", 
                      "koi_impact_err1", "koi_impact_err2", 
                      "koi_duration_err1", "koi_duration_err2", 
                      "koi_depth_err1", "koi_depth_err2",
                      "koi_prad_err1", "koi_prad_err2",
                      "koi_teq_err1", "koi_teq_err2",
                      "koi_insol_err1", "koi_insol_err2",
                      "koi_tce_plnt_num", "koi_tce_delivname",
                      "koi_steff_err1", "koi_steff_err2",
                      "koi_slogg_err1", "koi_slogg_err2",
                      "koi_srad_err1", "koi_srad_err2", # all columns above are plus/minus errors of their respective columns measurements, not important features
                      ] # merge with kepler name, id and oi name? 
columnsDropPHL = ["P_RADIUS","P_RADIUS_ERROR_MIN","P_RADIUS_ERROR_MAX", "P_MASS_ERROR_MIN", "P_MASS_ERROR_MAX", # missing values in these columns
                  "P_YEAR", "P_UPDATED", # not useful for habilitability
                  "P_PERIOD_ERROR_MIN", "P_PERIOD_ERROR_MAX",
                  "P_SEMI_MAJOR_AXIS_ERROR_MIN", "P_SEMI_MAJOR_AXIS_ERROR_MAX",
                  "P_ECCENTRICITY_ERROR_MIN", "P_ECCENTRICITY_ERROR_MAX",
                  "P_INCLINATION_ERROR_MIN", "P_INCLINATION_ERROR_MAX",
                  "P_OMEGA_ERROR_MIN", "P_OMEGA_ERROR_MAX",
                  "P_TPERI_ERROR_MIN", "P_TPERI_ERROR_MAX",
                  "P_IMPACT_PARAMETER", "P_IMPACT_PARAMETER_ERROR_MIN", 
                  "P_IMPACT_PARAMETER_ERROR_MAX", "P_TEMP_MEASURED", "P_GEO_ALBEDO", 
                  "P_GEO_ALBEDO_ERROR_MIN", "P_GEO_ALBEDO_ERROR_MAX", "P_DETECTION",
                  "P_DETECTION_MASS", "P_DETECTION_RADIUS", "P_ALT_NAMES", "P_ATMOSPHERE",
                  "S_DISTANCE_ERROR_MIN", "S_DISTANCE_ERROR_MAX",
                  "S_METALLICITY_ERROR_MIN", "S_METALLICITY_ERROR_MAX",
                  "S_MASS_ERROR_MIN", "S_MASS_ERROR_MAX", 
                  "S_RADIUS_ERROR_MIN", "S_RADIUS_ERROR_MAX",
                  "S_AGE_ERROR_MIN", "S_AGE_ERROR_MAX",
                  "S_TEMPERATURE_ERROR_MIN", "S_TEMPERATURE_ERROR_MAX", 
                  "S_DISC", "S_MAGNETIC_FIELD", 
                  "S_ALT_NAMES", "P_ESCAPE", "P_POTENTIAL", "P_GRAVITY", "P_DENSITY",
                  "S_TYPE_TEMP", "P_TYPE_TEMP",
                  "S_CONSTELLATION", "S_CONSTELLATION_ABR", "S_CONSTELLATION_ENG",
                  "S_RA_H", "S_RA_T", "S_DEC_T" # all columns above are plus/minus errors of their respective columns measurements or not important features that helps with habilitability 
                   ] # merge with p name (planet name) or s name (star name)?


# dropping columns and recreating csv's
dfGAIA = dfGAIA.drop(columns=columnsDropGAIA)
dfNASAExo = dfNASAExo.drop(columns=columnsDropNASAExo)
dfDropPHL = dfDropPHL.drop(columns=columnsDropPHL)


dfGAIA.to_csv('../data/droppeddata/GAIA.csv', index=False)
dfNASAExo.to_csv('../data/droppeddata/NASAExo.csv', index=False)
dfDropPHL.to_csv('../data/droppeddata/PHL.csv', index=False)

In [4]:
'''Step 3: Merge NASAExo and PHL'''

# Load preprocessed datasets
dfNASAExo = pd.read_csv('../data/droppeddata/NASAExo.csv')
dfPHL = pd.read_csv('../data/droppeddata/PHL.csv')

# Rename columns for consistency
dfPHL.rename(columns={"P_NAME": "planet_name", "S_NAME": "star_name"}, inplace=True)
dfNASAExo.rename(columns={"kepler_name": "planet_name"}, inplace=True)

# Perform merges
finalMerge = pd.merge(dfNASAExo, dfPHL, on="planet_name", how="left")
finalMerge.to_csv('../data/mergeddata/merged_NASAExo_PHL.csv', index=False)

In [8]:
'''Step 4: Merge GAIA and NASAExo_PHL'''

from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd

# Load datasets
NASAExo_PHL = pd.read_csv('../data/mergeddata/merged_NASAExo_PHL.csv')
GAIA = pd.read_csv('../data/droppeddata/GAIA.csv')

GAIA['ra'] = GAIA['ra'].round(4)
GAIA['dec'] = GAIA['dec'].round(4)

# Creating SkyCoord objects for matching
NASAExo_PHL_coords = SkyCoord(ra=NASAExo_PHL['ra'].values * u.degree, 
                              dec=NASAExo_PHL['dec'].values * u.degree, 
                              frame='icrs')
GAIA_coords = SkyCoord(ra=GAIA['ra'].values * u.degree, 
                       dec=GAIA['dec'].values * u.degree, 
                       frame='icrs')

# Transform NASAExo_PHL to ICRS
NASAExo_PHL_coords_icrs = NASAExo_PHL_coords.transform_to('icrs')

# Debug range of coordinates
print("NASAExo_PHL (ICRS) RA Range:", NASAExo_PHL_coords_icrs.ra.deg.min(), "to", NASAExo_PHL_coords_icrs.ra.deg.max())
print("NASAExo_PHL (ICRS) DEC Range:", NASAExo_PHL_coords_icrs.dec.deg.min(), "to", NASAExo_PHL_coords_icrs.dec.deg.max())
print("GAIA RA Range:", GAIA['ra'].min(), "to", GAIA['ra'].max())
print("GAIA DEC Range:", GAIA['dec'].min(), "to", GAIA['dec'].max())

# Perform SkyCoord Matching
idx, d2d, _ = GAIA_coords.match_to_catalog_sky(NASAExo_PHL_coords_icrs)
matches = d2d < 1 * u.arcsecond

# Filter matched pairs
GAIA_matches = GAIA[matches]
NASAExo_PHL_matches = NASAExo_PHL.iloc[idx[matches]]

# Merge matched datasets
mergedDF = pd.concat([NASAExo_PHL_matches.reset_index(drop=True), GAIA_matches.reset_index(drop=True)], axis=1)

# Export final merged dataset to CSV
mergedDF.to_csv('../data/mergeddata/GAIA_NASAExo_PHL_merged.csv', index=False)

NASAExo_PHL (ICRS) RA Range: 279.85272 to 301.72076
NASAExo_PHL (ICRS) DEC Range: 36.577381 to 52.33601
GAIA RA Range: 156.3226 to 316.6688
GAIA DEC Range: -64.3699 to 76.031
